In [16]:
#dependencies
import pandas as pd
from sportsreference.nba.teams import Teams
from scipy.stats import pearsonr

In [17]:
#pulling NBA team data from 2017-2019 for model training, adding year to each dataframe, and concatenating all 3 
NBA17 = Teams("2017").dataframes
NBA17 = NBA17.assign(Season=['2017'] * 30)
NBA18 = Teams("2018").dataframes
NBA18= NBA18.assign(Season=['2018'] * 30)
NBA19 = Teams("2019").dataframes
NBA19 = NBA19.assign(Season=['2019'] * 30)
NBA = [NBA17, NBA18, NBA19]
NBA = pd.concat(NBA)
#changing datatype of Season to match subsequent standings dataframe for future merge
NBA['Season'] = NBA['Season'].astype('int64')
NBA

,abbreviation,assists,blocks,defensive_rebounds,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempts,free_throw_percentage,free_throws,...,steals,three_point_field_goal_attempts,three_point_field_goal_percentage,three_point_field_goals,total_rebounds,turnovers,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,Season
GSW,GSW,2491,555,2873,7140,0.495,3532,1850,0.788,1457,...,785,2562,0.383,982,3643,1211,4578,0.557,2550,2017
HOU,HOU,2070,352,2746,7152,0.462,3305,2177,0.766,1667,...,670,3306,0.357,1181,3638,1240,3846,0.552,2124,2017
DEN,DEN,2077,323,2837,7194,0.469,3377,1986,0.774,1537,...,568,2365,0.368,870,3803,1226,4829,0.519,2507,2017
CLE,CLE,1858,327,2821,6963,0.470,3275,1913,0.748,1431,...,539,2779,0.384,1067,3587,1121,4184,0.528,2208,2017
WAS,WAS,1956,336,2671,7137,0.475,3388,1812,0.784,1421,...,693,2030,0.372,756,3514,1162,5107,0.515,2632,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MIA,MIA,1991,448,2879,7218,0.450,3251,1782,0.695,1238,...,627,2658,0.349,928,3800,1208,4560,0.509,2323,2019
CHI,CHI,1796,351,2799,7205,0.453,3266,1695,0.783,1328,...,603,2123,0.351,745,3517,1159,5082,0.496,2521,2019
NYK,NYK,1646,422,2811,7241,0.433,3134,1956,0.759,1484,...,557,2421,0.340,823,3668,1151,4820,0.479,2311,2019
CLE,CLE,1698,195,2619,7184,0.444,3189,1694,0.792,1342,...,534,2388,0.355,847,3498,1106,4796,0.488,2342,2019


In [18]:
#reading in standings file to merge on stats df
try:
    standings = pd.read_excel ('standings.xlsx')
except:
    standings=pd.read_excel('standings.xlsx', engine='openpyxl')
standings.head()

,abbreviation,W,L,W/L%,PS/G,PA/G,SRS,Season
0,TOR,59,23,0.720,111.7,103.9,7.29,2017
1,BOS,55,27,0.671,104.0,100.4,3.23,2017
2,PHI,52,30,0.634,109.8,105.3,4.30,2017
3,CLE,50,32,0.610,110.9,109.9,0.59,2017
4,IND,48,34,0.585,105.6,104.2,1.18,2017


In [19]:
df_merged = pd.merge(NBA, standings, on=['abbreviation', 'Season'], how='left')
df_merged.head()

,abbreviation,assists,blocks,defensive_rebounds,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempts,free_throw_percentage,free_throws,...,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,Season,W,L,W/L%,PS/G,PA/G,SRS
0,GSW,2491,555,2873,7140,0.495,3532,1850,0.788,1457,...,4578,0.557,2550,2017,58,24,0.707,113.5,107.5,5.79
1,HOU,2070,352,2746,7152,0.462,3305,2177,0.766,1667,...,3846,0.552,2124,2017,65,17,0.793,112.4,103.9,8.21
2,DEN,2077,323,2837,7194,0.469,3377,1986,0.774,1537,...,4829,0.519,2507,2017,46,36,0.561,110.0,108.5,1.57
3,CLE,1858,327,2821,6963,0.470,3275,1913,0.748,1431,...,4184,0.528,2208,2017,50,32,0.610,110.9,109.9,0.59
4,WAS,1956,336,2671,7137,0.475,3388,1812,0.784,1421,...,5107,0.515,2632,2017,43,39,0.524,106.6,106.0,0.53


In [20]:
#Testing the correlation between each team statistic and wins
corr_test = df_merged.copy()
corr_test = corr_test.drop(columns = ["abbreviation", "name", "L", "W/L%", "PS/G", "PA/G", "SRS"])

high_corr_cols = ["Season", "abbreviation"]

#Iterating the columns and printing mid-strength or above correlations
for col in corr_test.columns: 
    r = pearsonr(corr_test[col],corr_test["W"])
    
    if r[0] > .4 or r[0] < -0.4:
        high_corr_cols.append(col)
        # print(col)   
        # print(r)
        # print('-'*20)  

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [21]:
#Testing the correlation between each team statistic and rank
corr_test_rank = df_merged.copy()
corr_test_rank = corr_test_rank.drop(columns = ["abbreviation", "name", "L", "W/L%", "PS/G", "PA/G", "SRS"])

high_corr_rank_cols = ["Season", "abbreviation"]

#Iterating the columns and printing mid-strength or above correlations
for col in corr_test_rank.columns: 
    r = pearsonr(corr_test_rank[col],corr_test_rank["rank"])
    
    if r[0] > .4 or r[0] < -0.4:
        high_corr_rank_cols.append(col)
        print(col)   
        print(r)
        print('-'*20)  

assists
(-0.4398866800935304, 1.4370854446818109e-05)
--------------------
defensive_rebounds
(-0.4119133875486573, 5.489660850686179e-05)
--------------------
field_goal_percentage
(-0.6692763009397741, 5.556671057056261e-13)
--------------------
field_goals
(-0.6684114931518889, 6.100287848268057e-13)
--------------------
free_throw_attempts
(-0.4900654632233758, 9.456800480149415e-07)
--------------------
free_throws
(-0.5420294735122165, 3.439103959760273e-08)
--------------------
opp_field_goal_attempts
(-0.5053672447104094, 3.7711474809581905e-07)
--------------------
opp_two_point_field_goal_attempts
(-0.4653002952415415, 3.822350279209548e-06)
--------------------
points
(-0.8195893775683116, 5.332154518397784e-23)
--------------------
rank
(0.9999999999999999, 0.0)
--------------------
total_rebounds
(-0.4002184778253343, 9.290630819482241e-05)
--------------------
two_point_field_goal_percentage
(-0.6976120711413647, 2.1758383412931523e-14)
--------------------


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [11]:
NBA.columns

Index(['abbreviation', 'assists', 'blocks', 'defensive_rebounds',
       'field_goal_attempts', 'field_goal_percentage', 'field_goals',
       'free_throw_attempts', 'free_throw_percentage', 'free_throws',
       'games_played', 'minutes_played', 'name', 'offensive_rebounds',
       'opp_assists', 'opp_blocks', 'opp_defensive_rebounds',
       'opp_field_goal_attempts', 'opp_field_goal_percentage',
       'opp_field_goals', 'opp_free_throw_attempts',
       'opp_free_throw_percentage', 'opp_free_throws',
       'opp_offensive_rebounds', 'opp_personal_fouls', 'opp_points',
       'opp_steals', 'opp_three_point_field_goal_attempts',
       'opp_three_point_field_goal_percentage', 'opp_three_point_field_goals',
       'opp_total_rebounds', 'opp_turnovers',
       'opp_two_point_field_goal_attempts',
       'opp_two_point_field_goal_percentage', 'opp_two_point_field_goals',
       'personal_fouls', 'points', 'rank', 'steals',
       'three_point_field_goal_attempts', 'three_point_field_goa

In [13]:
NBA[['Season', 'abbreviation','rank']]

,Season,abbreviation,rank
GSW,2017,GSW,1
HOU,2017,HOU,2
DEN,2017,DEN,3
CLE,2017,CLE,4
WAS,2017,WAS,5
...,...,...,...
MIA,2019,MIA,26
CHI,2019,CHI,27
NYK,2019,NYK,28
CLE,2019,CLE,29


In [15]:
NBA17[['Season','rank']]

,Season,rank
GSW,2017,1
HOU,2017,2
DEN,2017,3
CLE,2017,4
WAS,2017,5
LAC,2017,6
BOS,2017,7
POR,2017,8
PHO,2017,9
TOR,2017,10
